In [1]:
import os
import spacy
import pandas as pd

nlp = spacy.load("en_core_web_sm")

In [2]:
def extract_ent(text):
  if type(text) != str:
    return pd.DataFrame()
  # first, get the doc
  doc = nlp(text)
  entList = []
  for ent in doc.ents:
        # for each entity we are saving the text, the number of words of the entity, 
        # the start and ending positions which will be used to check if the spans for Spacy and Stanza entities overlap
        # and the NER label
    entList.append({
      "text": ent.text,
      "n_word": len(ent.text.split(" ")),
      "start_pos": ent.start_char,
      "end_pos": ent.end_char,
      "label": ent.label_
    })
    # this information is saved in a df
  return pd.DataFrame(entList)


data_dir = os.getcwd()
data_dir = os.path.join(data_dir, "..", "..", "Data Directory")
df = pd.read_csv(os.path.join(data_dir, "physics_and_chemistry_nobel_laureate.csv"))

df.head()

,year,name,country,category,biography,clean_biography
0,1901,Wilhelm Röntgen,German Empire,Physics,Wilhelm Conrad Röntgen (; German pronunciation...,Wilhelm Conrad Röntgen german pronunciation ˈv...
1,1902,Hendrik Lorentz,Netherlands,Physics,Hendrik Antoon Lorentz (; 18 July 1853 – 4 Feb...,Hendrik Antoon Lorentz July February dutch phy...
2,1902,Pieter Zeeman,Netherlands,Physics,Pieter Zeeman (Dutch: [ˈzeːmɑn]; 25 May 1865 –...,Pieter Zeeman Dutch ˈzeːmɑn October dutch phys...
3,1903,Henri Becquerel,France,Physics,Antoine Henri Becquerel (; French pronunciatio...,Antoine Henri Becquerel french pronunciation b...
4,1903,Pierre Curie,France,Physics,"Pierre Curie ( KURE-ee, French: [pjɛʁ kyʁi]; 1...",Pierre Curie KURE ee French pjɛʁ kyʁi April fr...


In [3]:


df_entities_spacy = pd.DataFrame()
for index, row in df.iterrows():
# an empy df is going to be populated with some contextual information (text, category, subject) 
# as well as the information we get with SpaCy
  text = row['biography']
  category = row['category']
  subject = row["name"]
  df_ents = extract_ent(text)
  df_ents['biography'] = text
  df_ents['subject'] = subject
  df_ents['source_index'] = index
  df_ents["category"] = category
  # recursively adding up our master df with the enitty df
  df_entities_spacy = pd.concat([df_entities_spacy, df_ents], ignore_index = True)

In [4]:
df_entities_spacy.head()

,text,n_word,start_pos,end_pos,label,biography,subject,source_index,category
0,Wilhelm Conrad Röntgen,3.0,0.0,22.0,PERSON,Wilhelm Conrad Röntgen (; German pronunciation...,Wilhelm Röntgen,0,Physics
1,German,1.0,26.0,32.0,NORP,Wilhelm Conrad Röntgen (; German pronunciation...,Wilhelm Röntgen,0,Physics
2,27 March 1845,3.0,70.0,83.0,DATE,Wilhelm Conrad Röntgen (; German pronunciation...,Wilhelm Röntgen,0,Physics
3,10 February 1923,3.0,86.0,102.0,DATE,Wilhelm Conrad Röntgen (; German pronunciation...,Wilhelm Röntgen,0,Physics
4,German,1.0,110.0,116.0,NORP,Wilhelm Conrad Röntgen (; German pronunciation...,Wilhelm Röntgen,0,Physics


## Statistics for Spacy

In [5]:
# def computeStats(df_ents):

#     stats_count = df_ents['label'].value_counts().describe()

#     df_ents['length'] = df_ents['text'].apply(len)
#     stats_length = df_ents.groupby('label')['length'].describe()

#     return stats_count, stats_length



In [6]:
# saving the df to make the comparison between SpaCy and Stanza in the next notebook
spacy_ent_dir = os.path.join(data_dir,"df_entities_spacy.csv")
print(spacy_ent_dir)
df_entities_spacy.to_csv(spacy_ent_dir, index=False)

/Users/tunji/projects/unidlorraine/datascience-knowledge-graph/Part 2 - SpaCy and Stanza performance for NER/2.1 Named Entity Recognition/../../Data Directory/df_entities_spacy.csv
